In [1]:
# pip install --upgrade pip
# pip install git+https://github.com/PhilippNuspl/rec_sequences.git

In [2]:
#from ore_algebra import *
from rec_sequences.IntegerRelations import *

In [3]:
load('difference_field.sage')

In [4]:
#R.<t> = PolynomialRing(QQbar)
N.<t> = QQ[]
r =t^2 - t - 1
L=r.roots(ring=QQbar)
a = [i[0] for i in L]
#IntegerRelations.integer_relations(a)
a

[-0.618033988749895?, 1.618033988749895?]

In [5]:
M = r.splitting_field('b');M

Number Field in b with defining polynomial t^2 - t - 1

In [6]:
t = var('t')
sol = solve(t^2-t-1,t,algorithm='sympy');sol

[t == -1/2*sqrt(5) + 1/2, t == 1/2*sqrt(5) + 1/2]

In [7]:
lambda_1 = QQbar((1+sqrt(5))/2)
lambda_2 = QQbar((1-sqrt(5))/2)
print(a[0] == lambda_2)
print(a[1] == lambda_1)
IntegerRelations.integer_relations([lambda_1, lambda_2])

True
True


[-2 -2]

In [8]:
N.<t> = QQ[]
K.<b> = NumberField(t^2 - 5, embedding=QQbar(sqrt(5))) ## b=QQbar(sqrt(5))

In [9]:
print(b == QQbar(sqrt(5)))
print(lambda_1 == 1/2 + b/2)
print(lambda_2 == 1/2 - b/2)

True
True
True


In [10]:
def transition_matrix(A):
    """given a map sigma on the multivariate rational function field K(x) defined by simga(f(x)) = f(x*A),
    where K is a number field, x = [x_1, ..., x_n], A = (a_{i,j}) is a matrix in K^{n*n}, compute two matrices P, D 
    such that 
    
        f(x) is sigma-summable in K(x) if and only if f(P*x) is tau-summable in K(x), 
    
    where tau(f(x)) = (f(x*D)) and P, D are matirices in K^{n*n} such that A*(P^T) = (P^T)*D and D is digonal."""
    D, Q = A.eigenmatrix_right()
    P = Q.transpose()
    return P, D

In [11]:
t = polygen(QQ, 't')
K.<b> = NumberField(t^2 - 5, embedding=QQbar(sqrt(5)))
A = matrix(K, [[0,1],[1,1]])
print(A)
P, D = transition_matrix(A); P,D

[0 1]
[1 1]


(
[           1  1/2*b + 1/2]  [ 1/2*b + 1/2            0]
[           1 -1/2*b + 1/2], [           0 -1/2*b + 1/2]
)

In [12]:
def transition_map(f, P, x):
    """give a multivariate rational function f(x) and a matrix P, compute f(P*x)"""
    self, xx = make_in_multi_fraction_field(f,x, var_name = True)
    y = P*vector(xx)
    return self.subs({xx[i]: y[i] for i in range(len(x))}) 

In [13]:
Q = P.inverse()
print(Q)
R.<x1,x2> =  PolynomialRing(K)
S = R.fraction_field()
x = [x1,x2]
f = x1
print(transition_map(f, Q, x))
f = x2
print(transition_map(f, Q, x))


[-1/10*b + 1/2  1/10*b + 1/2]
[        1/5*b        -1/5*b]
(-1/10*b + 1/2)*x1 + (1/10*b + 1/2)*x2
(1/5*b)*x1 + (-1/5*b)*x2


In [14]:
#### Example 3.3 for Fibonacci sequences, in Hou, Wei, 2023 paper
#### Rational Solutions to the First Order Difference Equations in the Bivariate Difference Field
## define the difference automorphism and prepare the input
t = polygen(QQ, 't')
K.<b> = NumberField(t^2 - 5, embedding=QQbar(sqrt(5))) # b = QQbar(sqrt(5)), K = QQ(sqrt(5))
A = matrix(K, [[0,1],[1,1]])
P, D = transition_matrix(A) # D is the diagonalization of A
print(A)
print(D)
Q = P.inverse()
R.<x1,x2> =  PolynomialRing(K)
S = R.fraction_field()
x = [x1,x2]
F = x1/(x2*(x1+x2))
f = transition_map(F, Q, x); f # f(x) = F(P*x)

[0 1]
[1 1]
[ 1/2*b + 1/2            0]
[           0 -1/2*b + 1/2]


((-1/10*b + 1/2)*x1 + (1/10*b + 1/2)*x2)/((1/10*b + 1/10)*x1^2 - 1/5*x1*x2 + (-1/10*b + 1/10)*x2^2)

In [15]:
## decide whether f(x1,x2) is tau-summable in K(x1,x2),
## where tau(f(x1,x2)) = f(a_1*x1, a_2*x_2) and a = [a_1,a_2] = [1/2 + sqrt(5)/2,1/2 - sqrt(5)/2]
a = D.diagonal()
c = 1
pair = is_summable(f, x, a, c);pair

(True, ((1/2*b - 5/2))/((1/2*b - 1/2)*x1 + (-1/2*b + 1/2)*x2))

In [16]:
pair = is_summable(f, x, a, c)
g = pair[1]
f == c*sigma(g, x, a) - g  
## sigma(g, x, a) = g(a_1*x_1,..., a_n*x_n) if a = [a_1,...,a_n], x = [x_1,..., x_n]
## tau(g, x, a, i) = g(a_1^i*x_1,..., a_n^i*x_n) if a = [a_1,...,a_n], x = [x_1,..., x_n]

True

In [17]:
G = transition_map(g, P, x) # G(x) = g(P*x)
G

((1/2*b - 5/2))/((-1/2*b + 5/2)*x2)

In [18]:
#simplify((b - 5)/(-b + 5))
simplify((1/2*b - 5/2))/((-1/2*b + 5/2))

-1

In [19]:
#### test for bivariate rational functions: summable case
## define the rational function field S = QQ(sqrt(5))(x1,x2)
t = polygen(QQ, 't')
K.<b> = NumberField(t^2 - 5, embedding=QQbar(sqrt(5)))
R.<x1,x2> = PolynomialRing(K)
x = [*list(R.gens())]
S = R.fraction_field()
## or define the rational function field S in the following way such that S= QQ(sqrt(5))(x1)(x2)
# R.<x1> =  PolynomialRing(K)
# T.<x2> = R.fraction_field()['x2']
# x1,x2 = R.gen(),T.gen()
# x = [x1, x2]
# S = T.fraction_field()
h = S.random_element(3, 3)
# a = [0 for i in range(len(x))]
# while 0 in a:
#     a = [K.random_element() for i in range(len(x))]
# c = 0
# while c == 0:
#     c = K.random_element()
a = [1/2+1/2*b, 1/2-1/2*b]
c = 1
print("x=", x, "a=", a, "c=",c)
rel = IntegerRelations.integer_relations(a)
print("integer_relation = ", rel)
f = ((-1/10*b + 1/2)*x1 + (1/10*b + 1/2)*x2)/((1/10*b + 1/10)*x1^2 - 1/5*x1*x2 + (-1/10*b + 1/10)*x2^2)
#f =S(c * sigma(h, x, a) - h)
print("h = ", h)
print("f = ",f)

x= [x1, x2] a= [1/2*b + 1/2, -1/2*b + 1/2] c= 1
integer_relation =  [-2 -2]
h =  (-2*x2^2 + 1)/((-b - 1)*x2^2 + (6*b)*x2)
f =  ((-1/10*b + 1/2)*x1 + (1/10*b + 1/2)*x2)/((1/10*b + 1/10)*x1^2 - 1/5*x1*x2 + (-1/10*b + 1/10)*x2^2)


In [20]:
## check is_summable
pair = is_summable(f, x, a, c)
print(pair)
g = pair[1]
f == c*sigma(g, x, a) - g

(True, ((1/2*b - 5/2))/((1/2*b - 1/2)*x1 + (-1/2*b + 1/2)*x2))


True

In [21]:
L, Gp = orbital_partial_fraction(f,x2,x,a, group = True);L,Gp
## delete x2

([0, {x1 + (-1/2*b + 3/2)*x2: [[0, 1/2*b - 5/2, 1], [-1, -1/2*b + 5/2, 1]]}],
 {x1 + (-1/2*b + 3/2)*x2: [0, 1]})

In [22]:
## check orbital_partial_fraction
whole, parts = from_list_to_partial_fraction(L,x,a)
print(f == whole + sum(parts))
whole, parts


True


(0,
 [((-3/2*b - 5/2)*x2 - b*x1)/(x2^2 + ((1/2*b + 1/2)*x1)*x2 + (-1/2*b - 3/2)*x1^2)])

In [23]:
g, r_list =reduction(L[1], x, a, c)
g, r_list

(((1/2*b - 5/2))/((1/2*b - 1/2)*x1 + (-1/2*b + 1/2)*x2), [])

In [24]:
## check orbital_reduction
r = 0
for d, n, e in r_list:
    r = r + n/d**e
whole, parts = from_list_to_partial_fraction([0,L[1]],x,a)
sum(parts) == c * sigma(g, x, a) - g + r
print(g,r)

((1/2*b - 5/2))/((1/2*b - 1/2)*x1 + (-1/2*b + 1/2)*x2) 0


In [25]:
pair = additive_decomposition(f, x, a, c)
print(pair[1] == 0)
g = pair[0]
r = pair[1]
g,r

True


(((1/2*b - 5/2))/((1/2*b - 1/2)*x1 + (-1/2*b + 1/2)*x2), 0)

In [26]:
## check additive_decomposition
f == c*sigma(g, x, a) - g + r

True

In [27]:
#### test for bivariate rational functions: non-summable case
t = polygen(QQ, 't')
K.<b> = NumberField(t^2 - 5, embedding=QQbar(sqrt(5)))
R.<x1,x2> = PolynomialRing(K)
x = [*list(R.gens())]
S = R.fraction_field()
h = S.random_element(3, 5)
u = S.random_element(2, 2)
# a = [0 for i in range(len(x))]
# while 0 in a:
#     a = [K.random_element() for i in range(len(x))]
# c = 0
# while c == 0:
#     c = K.random_element()
a = [1/2 + 1/2*b,1/2-1/2*b]
c = 1
print("x=", x, "a=", a, "c=",c)
rel = IntegerRelations.integer_relations(a)
print("integer_relation = ", rel)
f =S(c*sigma(h, x, a) - h + u)
print("h = ", h)
print("u = ", u)
print("f = ", f)

x= [x1, x2] a= [1/2*b + 1/2, -1/2*b + 1/2] c= 1
integer_relation =  [-2 -2]
h =  ((-4*b + 1)*x1^2*x2 + (b + 1)*x1*x2^2 + (2*b + 23)*x2^3 + (11*b + 1)*x1^2 + (b + 2)*x1)/((-2*b - 1)*x1^3 + (9*b - 1)*x2^3 + (b + 1)*x1*x2 + (3*b + 1)*x1 + (2*b - 2))
u =  (-x1 + (b - 1))/((3*b + 2)*x1*x2 + 2)
f =  ((-929/2*b - 2355/2)*x1^6*x2^2 + (131*b + 265)*x1^5*x2^3 + (1612*b + 2250)*x1^4*x2^4 + (1209*b - 875)*x1^3*x2^5 + (-58*b + 590)*x1^2*x2^6 + (-29*b - 62)*x1^7 + (388*b + 1106)*x1^6*x2 + (38*b + 472)*x1^4*x2^3 + (730*b + 10682)*x1^3*x2^4 + (6*b + 578)*x1^2*x2^5 + (442*b - 902)*x1*x2^6 + (33*b + 83)*x1^6 + (11/2*b + 129/2)*x1^5*x2 + (964*b + 2308)*x1^4*x2^2 + (-212*b + 1084)*x1^3*x2^3 + (-1248*b + 1176)*x1^2*x2^4 + (92*b - 100)*x1*x2^5 + (1344*b - 3112)*x2^6 + (183*b + 291)*x1^5 + (514*b + 498)*x1^4*x2 + (214*b + 324)*x1^3*x2^2 + (1516*b - 792)*x1^2*x2^3 + (-448*b + 312)*x1*x2^4 + (-17*b - 75)*x1^4 + (330*b + 1134)*x1^3*x2 + (-12*b - 36)*x1^2*x2^2 + (656*b - 1736)*x1*x2^3 + (-30*b + 278)*x1^3 + (34*

In [28]:
## NOTE: It seems that the following procedure will run into an infinite loop.
## Because "IntegerRelations.integer_relations" could not output a result.
pair = additive_decomposition(f, x, a, c)
print(pair)
g = pair[0]
r = pair[1]
f == c*sigma(g, x, a) - g + r

(((-305/404*b + 819/404)*x1^5*x2^2 + (-53/19*b + 112/19)*x1^4*x2^3 + (7/19*b - 13/19)*x1^3*x2^4 + (43/82*b - 97/82)*x1^5*x2 + (127/19*b - 263/19)*x1^4*x2^2 + (963/3838*b - 2923/3838)*x1^3*x2^3 + (-2490/779*b + 5596/779)*x1^2*x2^4 + (92693/78679*b - 229991/78679)*x1^3*x2^2 + (85/1681*b - 207/1681)*x1^4 + (-672/779*b + 1514/779)*x1^3*x2 + (2906/1919*b - 6792/1919)*x1^2*x2^2 + (-10012/31939*b + 23512/31939)*x1*x2^3 + (24773/169781*b - 58431/169781)*x1^3 + (-16118/31939*b + 36958/31939)*x1^2*x2 + (-1996/31939*b + 4228/31939)*x1*x2^2 + (-40188/31939*b + 88456/31939)*x1^2 + (-164526/3225839*b + 402014/3225839)*x1*x2 + (-1135600/3225839*b + 2604144/3225839)*x1 + (-895592/3225839*b + 2028672/3225839))/((-b + 2)*x1^5*x2^2 + (113/19*b - 237/19)*x1^2*x2^5 + (7/19*b - 13/19)*x1^3*x2^3 + (31/19*b - 63/19)*x1^3*x2^2 + (34/19*b - 74/19)*x1^2*x2^2 + (292/1681*b - 632/1681)*x1^3 + (-33524/31939*b + 73572/31939)*x2^3 + (-1996/31939*b + 4228/31939)*x1*x2 + (-9100/31939*b + 19788/31939)*x1 + (-10216/31939

True

In [29]:
## problem?:
## a =  -1537600/64481201*b + 3399552/64481201
## IntegerRelations.integer_relations([a,-2*b + 6, -9*b + 2])

In [30]:
#R.<t> = PolynomialRing(QQbar)
N.<t> = QQ[]
r =t^3 - t^2 - t - 1  # characteristic polynomial of Tribonacci sequences
L=r.roots(ring=QQbar);L
a=[i[0] for i in L]
IntegerRelations.integer_relations(a)
a

[1.839286755214161?,
 -0.4196433776070806? - 0.6062907292071993?*I,
 -0.4196433776070806? + 0.6062907292071993?*I]

In [31]:
factor(r)

t^3 - t^2 - t - 1

In [32]:
p = t^6 + t^5 + 6*t^4 + t^3 + 24*t^2 + 35*t + 53
p.roots(ring=QQbar)[0][0]

-0.9196433776070805? - 2.264603124384899?*I

In [33]:
t = var('t')
p = t^6 + t^5 + 6*t^4 + t^3 + 24*t^2 + 35*t + 53
sol = solve(t^3-t^2-t-1,t,algorithm='sympy');sol

[t == 1/3*(3*sqrt(33) + 19)^(1/3) + 4/3/(3*sqrt(33) + 19)^(1/3) + 1/3,
 t == 1/6*I*sqrt(3)*(3*sqrt(33) + 19)^(1/3) - 1/6*(3*sqrt(33) + 19)^(1/3) - 2/3*I*sqrt(3)/(3*sqrt(33) + 19)^(1/3) - 2/3/(3*sqrt(33) + 19)^(1/3) + 1/3,
 t == -1/6*I*sqrt(3)*(3*sqrt(33) + 19)^(1/3) - 1/6*(3*sqrt(33) + 19)^(1/3) + 2/3*I*sqrt(3)/(3*sqrt(33) + 19)^(1/3) - 2/3/(3*sqrt(33) + 19)^(1/3) + 1/3]

In [34]:
M = r.splitting_field('b');M

Number Field in b with defining polynomial t^6 + t^5 + 6*t^4 + t^3 + 24*t^2 + 35*t + 53

In [35]:
#### Example for Tribonacci sequences
## define the difference automorphism and prepare the input
t = polygen(QQ, 't')
min_poly = t^6 + t^5 + 6*t^4 + t^3 + 24*t^2 + 35*t + 53
one_root = min_poly.roots(ring=QQbar)[0][0]
K.<b> = NumberField(min_poly, embedding=one_root) 
A = matrix(K, [[0,1,0],[0,0,1],[1,1,1]]).transpose()
print(A)
P, D = transition_matrix(A) # D is the diagonalization of A
print(D)
Q = P.inverse()
R.<x1,x2,x3> = PolynomialRing(K)
x = [*list(R.gens())]
S = R.fraction_field()
x = [x1,x2,x3]
IntegerRelations.integer_relations(a)

[0 0 1]
[1 0 1]
[0 1 1]
[         7/187*b^5 + 9/187*b^4 + 30/187*b^3 + 52/187*b^2 + 76/187*b + 376/187                                                                             0                                                                             0]
[                                                                            0           -1/143*b^5 - 4/143*b^4 + 8/143*b^3 - 42/143*b^2 - 7/143*b - 108/143                                                                             0]
[                                                                            0                                                                             0 -74/2431*b^5 - 49/2431*b^4 - 526/2431*b^3 + 38/2431*b^2 - 79/221*b - 621/2431]


[1 1 1]

In [36]:
#### Example: an identity involving the Tribonacci sequences
#### sum_{k=1}^n T_k = 1/2*T_{n+2} + 1/2*T_n - 1/2
a = D.diagonal()
c = 1
F = x1
f = S(transition_map(F, Q, x)); f # f(x) = F(P*x)

(-12/2057*b^5 - 13/2057*b^4 - 115/4114*b^3 - 205/4114*b^2 - 27/374*b + 152/2057)*x1 + (-1/1573*b^5 + 9/1573*b^4 - 49/3146*b^3 + 101/1573*b^2 - 3/143*b + 1591/3146)*x2 + (173/26741*b^5 + 16/26741*b^4 + 1164/26741*b^3 - 769/53482*b^2 + 453/4862*b + 22483/53482)*x3

In [37]:
pair = is_summable(f, x, a, c);pair

(True,
 (29/4114*b^5 + 47/4114*b^4 + 50/2057*b^3 + 81/2057*b^2 + 1/17*b + 315/2057)*x1 + (-15/3146*b^5 - 4/1573*b^4 - 5/1573*b^3 - 29/1573*b^2 - 19/286*b - 576/1573)*x2 + (-61/26741*b^5 - 475/53482*b^4 - 565/26741*b^3 - 560/26741*b^2 + 37/4862*b - 15347/53482)*x3)

In [38]:
g = pair[1]
f == c*sigma(g, x, a) - g

True

In [39]:
#### G_n = -1/2*T_{n} + 1/2*T_{n+2} = -1/2*T_{n} + 1/2*(T_{n+1} + T_{n} + T_{n-1}) 
#### so G_n = 1/2*T_{n+1} + 1/2*T_{n-1} and G_1 = 1
G = transition_map(g, P, x);G

-1/2*x1 + 1/2*x3